In [26]:
import ROOT

In [27]:
def linspace(start, end, steps):
    values = []
    for i in range(steps):
        values.append(start + i*(end-start)/(steps-1))
    return values

In [32]:
real_signal_mu = 60
real_signal_sigma = 20

model_signal_mu_spread_interval_length = 50 * 2
model_signal_mu_step_count = 5

model_signal_mus = linspace(real_signal_mu-model_signal_mu_spread_interval_length/2, real_signal_mu + model_signal_mu_spread_interval_length/2, model_signal_mu_step_count)
model_signal_sigma = real_signal_sigma

treshold_lower = 0
treshold_upper = 60
treshold_step_count = 5
treshold_values =  linspace(treshold_lower, treshold_upper, treshold_step_count)

noise_mu = 0
noise_sigma = 5

print(model_signal_mus)
print(treshold_values)

model_noise = ROOT.TF1("ElecNoiseBg", "TMath::Gaus(x, %f, %f)" % (noise_mu, noise_sigma), -100, 500)
model_noise.SetNpx(10000)


"""
# more paramaters
simulation_time = 1 #seconds
bunch_crossings_per_second = 40 * 1e6
collisions_per_bunch_crossing = en fördelning
particles_per_collision = en fördelning
particles_reaching_the_sensor = en fördelning vi inte känner till och inte är vår uppgift att känna till
 
"""

prng = ROOT.TRandom3(123)

for model_signal_mu in model_signal_mus:
    model_signal = ROOT.TF1("Signal", "TMath::Landau(x, %f, %f)" % (model_signal_mu, model_signal_sigma), -100, 500)
    model_signal.SetNpx(10000)
    for treshold_value in treshold_values:
        model_expected_signal_above = 1 - model_signal.Integral(-100, treshold_value)/model_signal.Integral(-100, 500)
        model_expected_noise_above = 1 - model_noise.Integral(-100, treshold_value)/model_noise.Integral(-100, 500)
        #print(model_expected_signal_above)
        #print(model_expected_noise_above)
        
        event_count = 100
        event_ratio = 0.5
        signal_count = int(round(event_count * event_ratio))
        noise_count = event_count - signal_count

        noise_above_count = 0
        signal_above_count = 0

        for i in range(signal_count):
            signal = prng.Landau(real_signal_mu, real_signal_sigma)
            if signal > treshold_value:
                signal_above_count += 1

        for i in range(noise_count):
            noise = prng.Gaus(noise_mu, noise_sigma)
            if noise > treshold_value:
                noise_above_count += 1
                
        total_above_count = noise_above_count + signal_above_count
        
        """
        Antag att vi vet:
            total_above_count
            event_ratio
            model_expected_signal_above
            model_expected_noise_above
        
        Hur ska vi räkna?
        Vi vill beräkna ett värde som är så nära signal_count som möjligt.
        """
        
        total_count = total_above_count/(event_ratio*model_expected_signal_above + (1-event_ratio)*model_expected_noise_above)
        computed_signal_count = total_count*event_ratio
        # Compare with signal_count which is the correct value
        print([model_signal_mu, treshold_value])
        print(computed_signal_count/signal_count)
        
        
        # Hur vill vi spara ner det här då?
        # Vi vill göra en snygg plott på något sätt.
        
        
        
        
        

[10.0, 35.0, 60.0, 85.0, 110.0]
[0.0, 15.0, 30.0, 45.0, 60.0]
[10.0, 0.0]
1.221345037406981
[10.0, 15.0]
1.5275759951686552
[10.0, 30.0]
1.782158943474961
[10.0, 45.0]
1.9732757289670946
[10.0, 60.0]
2.077757489156052
[35.0, 0.0]
0.9386587645117792
[35.0, 15.0]
1.156482200889652
[35.0, 30.0]
1.1796944085887775
[35.0, 45.0]
1.5464923740008505
[35.0, 60.0]
1.5937636259704047
[60.0, 0.0]
1.0400627831942728
[60.0, 15.0]
0.9847708641248015
[60.0, 30.0]
1.0131998667988535
[60.0, 45.0]
1.0500350772301361
[60.0, 60.0]
1.0892556065178258
[85.0, 0.0]
0.986666666667005
[85.0, 15.0]
0.9786794675978296
[85.0, 30.0]
0.9004690678852866
[85.0, 45.0]
0.812104157685512
[85.0, 60.0]
0.6761398713243492
[110.0, 0.0]
1.0399999999999998
[110.0, 15.0]
0.9986519217365641
[110.0, 30.0]
0.9599999992208796
[110.0, 45.0]
0.8600085814963303
[110.0, 60.0]
0.5411271039512136
